In [8]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

dataset = pd.read_csv('/Users/farrelmanazilin/Downloads/data/milk.csv')

# Hold-out Method (70%-30%)

In [4]:
datalabel = dataset.loc[:, ['Grade']]

xtrain, xtest, ytrain, ytest = train_test_split(dataset, datalabel, test_size=0.30, random_state=100)
test_label = np.array(xtest['Grade'])

newmin = 0
newmax = 1
mindata = xtrain.iloc[:, 0:-1].min()
maxdata = xtrain.iloc[:, 0:-1].max()
train_data = ((xtrain.iloc[:, 0:-1] - mindata) * (newmax - newmin) / (maxdata - mindata)) + newmin
test_data = ((xtest.iloc[:, 0:-1] - mindata) * (newmax - newmin) / (maxdata - mindata)) + newmin

kNN = KNeighborsClassifier(n_neighbors=3, weights='distance')
kNN.fit(train_data, ytrain.values.ravel())  
y_pred = kNN.predict(test_data)

accuracy = accuracy_score(ytest, y_pred)
error_ratio_holdout = 1 - accuracy

print("Error ratio untuk metode Hold-out (30-70%): {:.10f}".format(accuracy ))
print("Error ratio untuk metode Hold-out (30-70%): {:.10f}".format(error_ratio_holdout))

Error ratio untuk metode Hold-out (30-70%): 0.9937106918
Error ratio untuk metode Hold-out (30-70%): 0.0062893082


In [5]:
print(train_data)
print(test_data)

           pH  Temprature  Taste  Odor  Fat   Turbidity    Colour
255  0.584615    0.107143    1.0   0.0   1.0        0.0  0.333333
662  0.261538    0.071429    1.0   0.0   1.0        0.0  1.000000
899  0.553846    0.160714    0.0   0.0   1.0        0.0  0.666667
380  0.553846    0.071429    1.0   0.0   1.0        0.0  1.000000
954  0.553846    0.053571    1.0   0.0   1.0        0.0  1.000000
..        ...         ...    ...   ...   ...        ...       ...
802  0.553846    0.196429    0.0   1.0   1.0        1.0  0.666667
53   0.784615    0.571429    1.0   0.0   1.0        1.0  1.000000
350  0.538462    0.071429    1.0   0.0   0.0        0.0  1.000000
79   0.584615    0.000000    0.0   0.0   0.0        1.0  0.000000
792  0.000000    0.107143    1.0   1.0   1.0        1.0  1.000000

[741 rows x 7 columns]
            pH  Temprature  Taste  Odor  Fat   Turbidity    Colour
384   0.553846    0.196429    0.0   0.0   0.0        1.0  0.666667
288   0.400000    0.285714    0.0   1.0   1.0     

# kfold

In [12]:
kf = KFold(n_splits=10, random_state=0, shuffle=True)
p = 0
error_ratios = []
for train_index, test_index in kf.split(dataset):
    p=p+1
    xtrain = dataset.loc[train_index]
    xtest = dataset.loc[test_index]
    ytrain = xtrain.loc[:, ["Grade"]]
    ytest = xtest.loc[:, ['Grade']]

    mindata_fold = xtrain.iloc[:, 0:-1].min()
    maxdata_fold = xtrain.iloc[:, 0:-1].max()
    train_data_fold = ((xtrain.iloc[:, 0:-1] - mindata_fold) * (newmax - newmin) / (maxdata_fold - mindata_fold)) + newmin
    test_data_fold = ((xtest.iloc[:, 0:-1] - mindata_fold) * (newmax - newmin) / (maxdata_fold - mindata_fold)) + newmin

    kNN_fold = KNeighborsClassifier(n_neighbors=3, weights='distance')
    kNN_fold.fit(train_data_fold, ytrain.values.ravel())

    y_pred_fold = kNN_fold.predict(test_data_fold)

    accuracy_fold = accuracy_score(ytest, y_pred_fold)
    error_ratio_fold = 1 - accuracy_fold
    error_ratios.append(error_ratio_fold)
    
    print(f"Fold {p}:")
    print("Train data:")
    print(train_data_fold)
    print("Test data:")
    print(test_data_fold)
    print("Akurasi:")
    print(accuracy_fold)
    print("Error:")
    print(error_ratio_fold)
    
avg_error_ratio_kfold = np.mean(error_ratios)
print("Rata-rata Error ratio untuk metode K-Fold Cross-Validation (k=10): {:.10f}".format(avg_error_ratio_kfold))
print("Rata-rata Error akurasi untuk metode K-Fold Cross-Validation (k=10): {:.10f}".format(1-avg_error_ratio_kfold))
print("Hasil error Kfold\n",error_ratios)    

Fold 1:
Train data:
            pH  Temprature  Taste  Odor  Fat   Turbidity    Colour
0     0.553846    0.017857    1.0   0.0   1.0        0.0  0.933333
2     0.846154    0.642857    1.0   1.0   1.0        1.0  0.400000
3     1.000000    0.000000    1.0   1.0   0.0        1.0  1.000000
4     0.553846    0.053571    0.0   0.0   0.0        0.0  1.000000
5     0.553846    0.053571    1.0   1.0   1.0        1.0  1.000000
...        ...         ...    ...   ...   ...        ...       ...
1054  0.569231    0.196429    1.0   1.0   0.0        0.0  0.466667
1055  0.569231    0.071429    1.0   0.0   1.0        0.0  1.000000
1056  0.000000    0.107143    1.0   1.0   1.0        1.0  1.000000
1057  0.584615    0.160714    1.0   0.0   1.0        0.0  0.666667
1058  0.861538    0.375000    0.0   1.0   1.0        1.0  1.000000

[953 rows x 7 columns]
Test data:
            pH  Temprature  Taste  Odor  Fat   Turbidity    Colour
1     0.553846    0.035714    0.0   1.0   0.0        1.0  0.866667
14    0

# LOO

In [13]:
loo = LeaveOneOut()
error_ratios = []

mindata_LOO = dataset.iloc[:, 0:-1].min()
maxdata_LOO = dataset.iloc[:, 0:-1].max()
dataset_normalized = ((dataset.iloc[:, 0:-1] - mindata_LOO) * (newmax - newmin) / (maxdata_LOO - mindata_LOO)) + newmin

for train_index, test_index in loo.split(dataset_normalized):
    xtrain = dataset_normalized.iloc[train_index]
    xtest = dataset_normalized.iloc[test_index]
    ytrain = datalabel.iloc[train_index]
    ytest = datalabel.iloc[test_index]

    kNN_LOO = KNeighborsClassifier(n_neighbors=3, weights='distance')
    kNN_LOO.fit(xtrain, ytrain.values.ravel())

    y_pred_LOO = kNN_LOO.predict(xtest)

    accuracy_LOO = accuracy_score(ytest, y_pred_LOO)
    error_ratio_LOO = 1 - accuracy_LOO

    error_ratios.append(error_ratio_LOO)

    print(f"LOO {p}:")
    print("Train data:")
    print(train_data_LOO)
    print("Test data:")
    print(test_data_LOO)
    print("Akurasi:")
    print(accuracy_LOO)
    print("Error:")
    print(error_ratio_LOO)

avg_error_ratio_LOO = np.mean(error_ratios)
print("Rata-rata Error ratio untuk metode Leave-One-Out (LOO) Cross-Validation: {:.10f}".format(avg_error_ratio_LOO))
print("Rata-rata Akurasi ratio untuk metode Leave-One-Out (LOO) Cross-Validation: {:.10f}".format(1-avg_error_ratio_LOO))
#print("Hasil error LOO\n",error_ratios) 

NameError: name 'LeaveOneOut' is not defined